Вариационная модель звезды

Как параметр задана масса звезды M.
Звезда разбита на N сферических слоев одинаковой массы M/N
Также задано распределение водорода по слоям.
Имеется N+1 точка - расстояния от центра до i-го слоя.
Неизвестными являются массивы расстояний r[i] и температур t[i]

Date of creation: 2.07.2020

Last updated: 2.07.2020

In [95]:
import math
import numpy as np
import pandas as pd

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

In [96]:
h = 1.054E-27 # reduced Planck constant
c = 3e10 # speed of light
G = 6.67E-8 # gravitational constant
kB = 1.381E-16 # Boltzmann constant
m_prot = 1.67E-24 # масса протона
m_elec = 9.11E-28 # масса электрона
alpha = 137 # e2/h/c - постоянная тонкой структуры
e2 = h*c/alpha
r_elec = e2/m_elec/c/c # классический радиус электрона
pi = math.pi # 3.14...
sigmaT = 8*pi/3*r_elec*r_elec # сечение томсоновского рассеяния на электронах
kappaT = sigmaT/m_prot
sigma = pow(pi, 2)*pow(kB, 4)/60*pow(h, -3)*pow(c, -2) # sigma*T^4
a = pow(pi, 2)*pow(kB, 4)/15*pow(h, -3)*pow(c, -3)
gamma = 5/3
year = 365.25*24*3600 # seconds in a year
eVolt = 1.6e-12 # ergs
dEpp = (26.23 - 0.5)*1e6*eVolt # energy output in (4p -> He) reaction minus neutrinos energy

In [97]:
M_sol = 1.99e33 # solar mass
R_sol = 6.96e10 # solar radius
L_sol = 3.85e33 # solar luminosity
Teff = pow(L_sol/4/pi/sigma/R_sol/R_sol, 1/4) # 5780 solar surface temperature

In [98]:
dimM = 200 # число сферических слоев
M = M_sol
dm = M/dimM

In [99]:
r1 = np.zeros(dimM) # расстояния - из модели
t1 = np.zeros(dimM) # температуры - из модели
d1 = np.zeros(dimM) # плотности

In [100]:
df1 = pd.read_csv('../Zeld/1epoch00.csv')
df1

,Unnamed: 0,Mass,Radius,Temperature,Density,Luminosity,Hydrogen,Opacity,Convection
0,0,9.947406e+30,0.000000e+00,1.331000e+07,94.427674,2.937955e+31,0.732,1.342981,0.002404
1,1,1.989481e+31,3.737520e+09,1.286046e+07,89.581399,1.591017e+32,0.732,1.389733,0.000000
2,2,2.984222e+31,4.301280e+09,1.272100e+07,87.504916,2.591621e+32,0.732,1.412782,0.000000
3,3,3.978962e+31,4.746720e+09,1.260008e+07,85.728836,3.514368e+32,0.732,1.429947,0.000000
4,4,4.973703e+31,5.136480e+09,1.248738e+07,84.121420,4.381412e+32,0.732,1.443379,0.000000
...,...,...,...,...,...,...,...,...,...
195,195,1.949692e+33,4.785000e+10,2.058934e+06,0.219782,2.438247e+33,0.732,20.723697,0.000000
196,196,1.959639e+33,4.978488e+10,1.805729e+06,0.172056,2.438247e+33,0.732,20.415850,0.000000
197,197,1.969586e+33,5.218608e+10,1.489402e+06,0.127010,2.438247e+33,0.732,18.836202,0.000000
198,198,1.979534e+33,5.547120e+10,1.056482e+06,0.083605,2.438247e+33,0.732,15.651353,0.775424


In [102]:
x1 = df1['Hydrogen']
r1 = df1['Radius']
t1 = df1['Temperature']
d1 = df1['Density']


In [103]:
r = np.zeros(dimM+1) # расстояния - заданы как 0-е приближение
t = np.zeros(dimM+1) # температуры - заданы как 0-е приближение
d = np.zeros(dimM+1) # плотности
l = np.zeros(dimM+1) # светимости
x = np.zeros(dimM+1) # содержания водорода - заданы
k = np.zeros(dimM+1) # непрозрачности
e = np.zeros(dimM+1) # энерговыделения
p = np.zeros(dimM+1) # давления
a = np.zeros(dimM) # градиент давления
da = np.zeros(dimM) # несоответсвие градиента давления
b = np.zeros(dimM) # градиент температуры
db = np.zeros(dimM) # несоответствие градиента температуры 

In [104]:
# Уравнение состояния 
def Pressure(den, T, X, Y, Z) :
    mu = 1/(2*X + 3/4*Y + 1/2*Z)
    return den/m_prot/mu*kB*T

In [105]:
# Зельдович, стр 70, 71:
# Скорость энерговыделения в p-p реакции [эрг/с/г] деленная на X
def E0pp(den, T, X) :
    T0 = (1e6, 5e6, 10e6, 15e6, 20e6, 30e6)
    e0 = (4e-9, 1.8e-3, 6.8e-2, 0.377, 1.09, 4.01)
    n = (10.6, 5.95, 4.60, 3.95, 3.64, 3.03)
    found = False
    for i in range (len(T0) - 1) :
        if T < pow(T0[i]*T0[i+1], 0.5) :
            found = True
            break;
        if not found :
            i = len(T0) - 1
    # print('i=%d T0=%.1e e0=%.3e n=%.2f' % (i, T0[i], e0[i], n[i]))
    return den*X*e0[i]*pow(T/T0[i], n[i])

# Скорость энерговыделения в CNO цикле [эрг/с/г] деленная на X
def E0cno(den, T, XCNO) :
    T0 = (6e6, 10e6, 15e6, 20e6, 30e6, 50e6, 100e6)
    e0 = (9e-10, 3.4e-4, 1.94, 4.5e2, 4.1e5, 6.2e8, 1.9e12)
    n = (27.3, 22.9, 19.9, 18.0, 15.6, 13.6, 10.2)
    found = False
    for i in range (len(T0) - 1) :
        if T < pow(T0[i]*T0[i+1], 0.5) :
            found = True
            break;
    if not found :
        i = len(T0) - 1 
    # print('i=%d T0=%.1e e0=%.3e n=%.2f' % (i, T0[i], e0[i], n[i]))
    return den*XCNO*e0[i]*pow(T/T0[i], n[i])

# Полное энерговыделение - домножаем на X.
def Etot(den, T, X, Y, Z) :
    return X*(E0pp(den, T, X) + E0cno(den, T, Z))
# For best fit to SSM18:
#     return 1.3*E0pp(den, T, X) + 3.5*E0cno(den, T, Z)

In [106]:
def MU(X, Y, Z) :
    return  1/(2*X + 3/4*Y + 1/2*Z)

# Табличные данные для функции непрозрачности
Tst = [2120000.,  2720000.,  3400000.,  4640000.,  6030000.,
    7270000.,  8000000.,  8770000.,  9600000., 10200000., 10800000.,
   11400000., 12400000., 13800000., 14800000., 15500000.]
Kst = [9.94082840e+01, 3.60946746e+01,
   1.03492885e+01, 1.64912281e+00, 4.35865504e-01, 1.73913043e-01,
   1.08597285e-01, 7.14285714e-02, 4.93506494e-02, 3.93873085e-02,
   3.17164179e-02, 2.53164557e-02, 1.77439797e-02, 1.20259019e-02,
   8.96191187e-03, 7.03774792e-03]
def kappa(den, T, X, Y, Z) :
    if T <= Tst[0] :
        return den*Kst[0]*T/Tst[0]
    for i in range(len(Tst) - 1) :
        if T <= Tst[i+1] :
            return den*(Kst[i] + (T - Tst[i])/(Tst[i+1] - Tst[i])*(Kst[i+1] - Kst[i]))
    return den*Kst[-1]

In [113]:
x[0] = x1[0]
t[0] = t1[0]
for i in range(dimM):
    x[i+1] = x1[i]
    t[i+1] = t1[i]
for i in range(dimM-1):
    r[i] = r1[i]
r[dimM] = r[dimM-1] + dm/4/pi/pow(r[dimM-1], 2)/d1[dimM-1]

In [118]:
d[0] = 3*dm/4/pi/pow(r[1], 3)
for i in range(1, dimM):
    d[i] = dm/2/pi/r[i]/r[i]/(r[i+1] - r[i-1])
d[dimM] = 0.

for i in range(0, dimM+1):
    e[i] = dm*x[i]*Etot(d[i], t[i], x[i], 1 - x[i] -Z, Z)
    p[i] = Pressure(d[i], t[i], x[i], 1 - x[i] -Z, Z)
    k[i] = kappa(d[i], t[i], x[i], 1 - x[i] -Z, Z)
    
l[0] = 0.
for i in range(1, dimM+1):
    l[i] += e[i]   

Условие гидростатики

In [126]:
A = kB/m_prot
for i in range(1, dimM):
    mu = MU(x[i], 1 - x[i] -Z, Z)
    a[i] = -G*M*i/dimM*d[i]*(r[i+1] - r[i])/pow(r[i], 2)
#     da[i] = (d[i] - d[i-1])*t[i] + d[i]*(t[i] - t[i-1]) - a[i]
    da[i] = (p[i+1] - p[i-1])/2 - a[i]

Условие переноса энергии

In [133]:
for i in range(1, dimM):
    rad = -3/16*k[i]*d[i]*l[i+1]*(r[i] - r[i-1])/sigma/pow(t[i], 3)/4/pi/pow(r[i], 2)
    mu = MU(x[i], 1 - x[i] -Z, Z)
    conv = -0.4*mu/A*G*i/dimM*M/pow(r[i], 2)*(r[i] - r[i-1])
    if abs(rad) > abs(conv):
        b[i] = rad
    else:
        b[i] = conv
    db[i] = (t[i+1] -t[i-1])/2 -b[i]

In [134]:

db[1:]/b[1:]

array([-0.56559166,  1.49882255,  0.13508319,  0.02217305, -0.01740574,
       -0.05655837, -0.07626369, -0.10252174, -0.09407193, -0.12187078,
       -0.09958513, -0.14479681, -0.10707176, -0.1580717 , -0.14899946,
       -0.15529813, -0.16228201, -0.1835764 , -0.16903565, -0.17016937,
       -0.17249306, -0.20991556, -0.15824225, -0.21722476, -0.18400193,
       -0.18823112, -0.21238835, -0.21522217, -0.21727983, -0.20009106,
       -0.22404918, -0.2277014 , -0.21123645, -0.25668209, -0.22074924,
       -0.24727321, -0.25249665, -0.23519555, -0.27995311, -0.24111837,
       -0.2664989 , -0.27046276, -0.27471769, -0.27927961, -0.28416257,
       -0.28937882, -0.27359582, -0.32202518, -0.2846456 , -0.30822846,
       -0.30897375, -0.32963786, -0.28926398, -0.31267241, -0.31490152,
       -0.29542627, -0.34233026, -0.3243683 , -0.32808363, -0.33218728,
       -0.31511083, -0.36262128, -0.34713908, -0.3316444 , -0.37920294,
       -0.3653168 , -0.34935919, -0.39076814, -0.37039829, -0.34

In [135]:
sa = 0
sb = 0
for i in range(1, dimM):
    sa += pow(da[i]/a[i], 2)
    sb += pow(db[i]/b[i], 2)
sa, sb    

(2718.252863920211, 34.14039871840519)

In [136]:
import SSM18

ModuleNotFoundError: No module named 'SSM18'